In [56]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from scipy import stats
import numpy as np
from datetime import datetime, timedelta

In [57]:
# 
file_loc0 = "../../ADNI_data/"
df_tau=pd.read_csv(file_loc0 + 'UCBERKELEY_TAU_6MM_23Apr2024.csv')
df_NPIQ=pd.read_csv(file_loc0 + 'NPIQ_05Mar2024.csv')
df_DX = pd.read_csv(file_loc0 + 'DXSUM_PDXCONV_ADNIALL_Diagnostic Summary_standardised.csv')
df_tau.shape

(1638, 338)

In [58]:
df_Fpet = df_tau.copy()

In [59]:
#convert to data time for manipulation
df_Fpet['SCANDATE_dt']=pd.to_datetime(df_Fpet.SCANDATE, format='%Y-%m-%d')

df_NPIQ['VISDATE_dt']=pd.to_datetime(df_NPIQ.VISDATE, format='%Y-%m-%d')
df_NPIQ['USERDATE_dt']=pd.to_datetime(df_NPIQ.USERDATE, format='%Y-%m-%d')

df_DX['EXAMDATE_dt']=pd.to_datetime(df_DX.EXAMDATE, format='%m/%d/%Y')

In [60]:
df_NPIQ.NPID.value_counts()

 0.0    5525
 1.0    1494
 2.0      20
-1.0      18
Name: NPID, dtype: int64

In [61]:
df_Fpet_selcol_lt  = ['RID','SCANDATE','SCANDATE_dt','qc_flag']

In [62]:
df_Fpet_selcol=df_Fpet[df_Fpet_selcol_lt].sort_values(by=['RID','SCANDATE_dt'])
df_Fpet_selcol.head()

,RID,SCANDATE,SCANDATE_dt,qc_flag
0,21,2018-02-02,2018-02-02,2
1,31,2018-04-24,2018-04-24,2
2,31,2019-04-23,2019-04-23,2
3,56,2018-02-20,2018-02-20,2
4,56,2019-01-10,2019-01-10,2


In [63]:
df_Fpet_selcol[['RID','SCANDATE']].drop_duplicates().shape

(1638, 2)

In [64]:
df_NPIQ_selcol_lt=['RID','Phase','VISCODE2','NPID','VISDATE','VISDATE_dt','USERDATE','USERDATE_dt']

In [65]:
df_NPIQ[df_NPIQ['VISDATE_dt']>(df_Fpet_selcol.SCANDATE_dt.min()-timedelta(days=365))].shape,df_NPIQ.shape

((668, 41), (7093, 41))

In [66]:
#merging on RID
df_Fpet_npiq=df_NPIQ[(df_NPIQ['NPID']>=0)&(df_NPIQ['VISDATE_dt']>(df_Fpet_selcol.SCANDATE_dt.min()-timedelta(days=365)))][['RID','VISCODE2','USERDATE_dt','VISDATE_dt','NPID']].merge(df_Fpet_selcol[['RID','SCANDATE_dt']].drop_duplicates()).sort_values(by=['RID','SCANDATE_dt'])
df_Fpet_npiq.shape,df_Fpet_selcol[['RID','SCANDATE_dt']].shape,df_NPIQ.shape,df_NPIQ[df_NPIQ['VISDATE_dt']>(df_Fpet_selcol.SCANDATE_dt.min()-timedelta(days=182))].shape

((1079, 6), (1638, 2), (7093, 41), (654, 41))

In [67]:
df_Fpet_npiq[['RID','SCANDATE_dt']].drop_duplicates().shape

(678, 2)

In [68]:
df_Fpet_npiq['diff_dt']=df_Fpet_npiq['VISDATE_dt']-df_Fpet_npiq['SCANDATE_dt']
df_Fpet_npiq['diff_dt_abs']=df_Fpet_npiq['diff_dt'].abs()

In [69]:
# df_Fpet_npiq.diff_dt.value_counts()

In [70]:
df_Fpet_selcol_62days=df_Fpet_npiq[ (df_Fpet_npiq['diff_dt_abs'] < '365 days')] # 182
df_Fpet_selcol_62days.shape

(252, 8)

In [71]:
df_Fpet_selcol_62days[['RID','SCANDATE_dt']].drop_duplicates().shape

(247, 2)

In [72]:
df_Fpet_selcol_62days.head()

,RID,VISCODE2,USERDATE_dt,VISDATE_dt,NPID,SCANDATE_dt,diff_dt,diff_dt_abs
1039,89,m180,2022-08-17,2021-02-19,0.0,2020-08-25,178 days,178 days
10,96,m144,2018-03-06,2018-03-06,0.0,2017-03-16,355 days,355 days
16,96,m168,2020-03-19,2020-03-16,0.0,2021-03-02,-351 days,351 days
864,205,m174,2021-05-01,2021-05-01,0.0,2021-10-05,-157 days,157 days
639,337,m168,2020-05-21,2020-05-21,0.0,2021-05-19,-363 days,363 days


In [73]:
df_Fpet_selcol_62days_first=df_Fpet_selcol_62days.merge(pd.DataFrame(df_Fpet_selcol_62days[['RID','SCANDATE_dt','diff_dt_abs']].groupby(['RID','SCANDATE_dt']).diff_dt_abs.min()).reset_index(),on=['RID','SCANDATE_dt','diff_dt_abs']).sort_values(by='diff_dt')
df_Fpet_selcol_62days_first.shape

(247, 8)

In [74]:
# df_Fpet_selcol_62days_first

In [75]:
df_Fpet_selcol_62days_first[['RID','SCANDATE_dt']].drop_duplicates().shape

(247, 2)

In [76]:
df_DX_adapt = df_DX[['RID','VISCODE2','EXAMDATE_dt','DX']].drop_duplicates().sort_values(by=['RID','EXAMDATE_dt']).rename(columns={'VISCODE2':'VISCODE2_dx'})
df_DX_adapt.shape

(13491, 4)

In [77]:
# Merging all three
df_Fpet_npiq_DX=df_Fpet_selcol_62days_first.merge(df_DX_adapt[df_DX_adapt['EXAMDATE_dt']>(df_Fpet_selcol_62days_first.SCANDATE_dt.min()-timedelta(days=365))]).sort_values(by=['RID','SCANDATE_dt'])
df_Fpet_DX=df_Fpet_selcol.merge(df_DX_adapt[df_DX_adapt['EXAMDATE_dt']>(df_Fpet_selcol.SCANDATE_dt.min()-timedelta(days=182))]).sort_values(by=['RID','SCANDATE_dt'])

df_Fpet_npiq_DX.shape,df_Fpet_DX.shape

((848, 11), (6283, 7))

In [78]:
df_Fpet_npiq_DX['diff_dx_dt']=df_Fpet_npiq_DX['EXAMDATE_dt']-df_Fpet_npiq_DX['SCANDATE_dt']
df_Fpet_DX['diff_dx_dt']=df_Fpet_DX['EXAMDATE_dt']-df_Fpet_DX['SCANDATE_dt']

In [79]:
df_Fpet_npiq_DX['diff_dx_dt_abs']=df_Fpet_npiq_DX['diff_dx_dt'].abs()
df_Fpet_DX['diff_dx_dt_abs']=df_Fpet_DX['diff_dx_dt'].abs()


In [80]:
df_Fpet_DX_62days=df_Fpet_DX[ (df_Fpet_DX['diff_dx_dt_abs'] < '182 days')].sort_values(by='RID')

df_Fpet_npiq_DX_62days=df_Fpet_npiq_DX[ (df_Fpet_npiq_DX['diff_dx_dt_abs'] < '365 days')].sort_values(by='RID')
df_Fpet_npiq_DX_62days.shape,df_Fpet_DX_62days.shape

((358, 13), (2129, 9))

In [81]:
df_Fpet_DX_62days_first=df_Fpet_DX_62days.merge(pd.DataFrame(df_Fpet_DX_62days[['RID','SCANDATE_dt','diff_dx_dt_abs']].groupby(['RID','SCANDATE_dt']).diff_dx_dt_abs.min()).reset_index(),on=['RID','SCANDATE_dt','diff_dx_dt_abs']).sort_values(by='diff_dx_dt')
df_Fpet_DX_62days_first.shape

(1566, 9)

In [82]:
df_Fpet_npiq_DX_62days['diff_ED_VD_dt']=df_Fpet_npiq_DX_62days['VISDATE_dt']-df_Fpet_npiq_DX_62days['EXAMDATE_dt']
df_Fpet_npiq_DX_62days.shape

(358, 14)

In [83]:
df_Fpet_npiq_DX_62days_final1=df_Fpet_npiq_DX_62days[(df_Fpet_npiq_DX_62days['diff_ED_VD_dt']> '-365 days') & (df_Fpet_npiq_DX_62days['diff_ED_VD_dt']<'365 days')].sort_values(by='diff_ED_VD_dt')
df_Fpet_npiq_DX_62days_final1.shape

(191, 14)

In [84]:
df_Fpet_npiq_DX_62days_final3=df_Fpet_npiq_DX_62days_final1.merge(pd.DataFrame(df_Fpet_npiq_DX_62days_final1[['RID','SCANDATE_dt','diff_dx_dt_abs']].groupby(['RID','SCANDATE_dt']).diff_dx_dt_abs.min()).reset_index(),on=['RID','SCANDATE_dt','diff_dx_dt_abs']).sort_values(by='diff_ED_VD_dt')
df_Fpet_npiq_DX_62days_final3.shape

(183, 14)

In [85]:
# df_Fpet_npiq_DX_62days_final3

In [86]:
df_Fpet_npiq_DX_62days_final3[['RID','SCANDATE_dt']].drop_duplicates().shape

(183, 2)

In [87]:
df_Fpet_npiq_DX_62days_final3.head()

,RID,VISCODE2,USERDATE_dt,VISDATE_dt,NPID,SCANDATE_dt,diff_dt,diff_dt_abs,VISCODE2_dx,EXAMDATE_dt,DX,diff_dx_dt,diff_dx_dt_abs,diff_ED_VD_dt
0,6069,m36,2020-10-09,2020-10-01,0.0,2021-09-08,-342 days,342 days,m48,2021-09-30,NL,22 days,22 days,-364 days
1,4441,m96,2021-09-20,2020-04-10,0.0,2021-04-08,-363 days,363 days,m108,2021-04-08,NL,0 days,0 days,-363 days
2,6547,m36,2021-09-30,2021-09-30,0.0,2022-09-29,-364 days,364 days,m48,2022-09-28,NL,-1 days,1 days,-363 days
3,6094,m42,2021-02-22,2021-02-22,0.0,2022-02-15,-358 days,358 days,m54,2022-02-15,NL,0 days,0 days,-358 days
4,5228,m90,2020-12-04,2020-11-30,0.0,2021-11-18,-353 days,353 days,m102,2021-11-23,NL,5 days,5 days,-358 days


In [88]:
df_Fpet_npiq_DX_62days_final3['diff_ud-vd']=df_Fpet_npiq_DX_62days_final3['USERDATE_dt']-df_Fpet_npiq_DX_62days_final3['VISDATE_dt']

In [89]:
df_Fpet_npiq_DX_62days_final3_Fpet=df_Fpet_npiq_DX_62days_final3[['RID','SCANDATE_dt','VISCODE2_dx','DX','NPID']].merge(df_Fpet,on=['RID','SCANDATE_dt'])
df_Fpet_npiq_DX_62days_final3_Fpet.shape

(183, 342)

In [90]:
df_Fpet_npiq_DX_62days_final3_Fpet.NPID.value_counts()

0.0    170
1.0     13
Name: NPID, dtype: int64

In [91]:
df_Fpet_DX_62days_first.DX.value_counts()

NL           848
MCI          507
AD           203
MCI to AD      4
MCI to NL      2
AD to MCI      1
NL to MCI      1
Name: DX, dtype: int64

In [92]:
df_Fpet_DX_62days_first.shape

(1566, 9)

In [93]:
df_Fpet_DX_62days_first.head()

,RID,SCANDATE,SCANDATE_dt,qc_flag,VISCODE2_dx,EXAMDATE_dt,DX,diff_dx_dt,diff_dx_dt_abs
797,6206,2019-11-12,2019-11-12,2,m12,2019-05-16,MCI,-180 days,180 days
535,5169,2017-11-16,2017-11-16,2,m48,2017-05-25,NL,-175 days,175 days
156,4028,2016-01-12,2016-01-12,2,m48,2015-07-23,NL,-173 days,173 days
1219,6618,2019-07-23,2019-07-23,2,bl,2019-02-04,MCI,-169 days,169 days
633,6027,2017-11-07,2017-11-07,2,sc,2017-05-23,NL,-168 days,168 days


In [94]:
df_Fpet_DX_62days_first_Fpet=df_Fpet_DX_62days_first[['RID','SCANDATE_dt','VISCODE2_dx','DX']].merge(df_Fpet,on=['RID','SCANDATE_dt'])
df_Fpet_DX_62days_first_Fpet.shape

(1566, 341)

In [95]:
df_Fpet_DX_62days_first_Fpet.RID.nunique()

919

In [96]:
df_Fpet_DX_62days_first_Fpet.RID.value_counts().value_counts()

1    507
2    242
3    114
4     47
5      9
Name: RID, dtype: int64

In [97]:
df_Fpet_DX_62days_first_Fpet.groupby('DX')['CTX_ENTORHINAL_SUVR'].count()

DX
AD           201
AD to MCI      1
MCI          505
MCI to AD      4
MCI to NL      2
NL           843
NL to MCI      1
Name: CTX_ENTORHINAL_SUVR, dtype: int64

In [98]:
df_Fpet_DX_62days_first_Fpet[df_Fpet_DX_62days_first_Fpet['DX'].isin(['NL','MCI','AD'])].groupby('DX')['CTX_ENTORHINAL_SUVR'].mean()

DX
AD     1.566701
MCI    1.321861
NL     1.174024
Name: CTX_ENTORHINAL_SUVR, dtype: float64

In [99]:
df_Fpet_DX_62days_first_Fpet[df_Fpet_DX_62days_first_Fpet['DX'].isin(['NL','MCI','AD'])].groupby('DX')['RIGHT_ACCUMBENS_AREA_SUVR'].mean()

DX
AD     1.458808
MCI    1.350216
NL     1.312368
Name: RIGHT_ACCUMBENS_AREA_SUVR, dtype: float64

In [100]:
df_Fpet_DX_62days_first_Fpet[df_Fpet_DX_62days_first_Fpet['DX'].isin(['NL','MCI','AD'])].groupby('DX')['RIGHT_AMYGDALA_SUVR'].count()

DX
AD     198
MCI    459
NL     807
Name: RIGHT_AMYGDALA_SUVR, dtype: int64

In [101]:
df_Fpet_DX_62days_first_Fpet[df_Fpet_DX_62days_first_Fpet['DX'].isin(['NL','MCI','AD'])].groupby('DX')['RIGHT_AMYGDALA_SUVR'].mean()

DX
AD     1.613601
MCI    1.379532
NL     1.229020
Name: RIGHT_AMYGDALA_SUVR, dtype: float64

In [102]:
# df_Fpet_DX_62days_first_Fpet.RID.value_counts().head(9).index.values

In [103]:
# df_Fpet_DX_62days_first_Fpet[df_Fpet_DX_62days_first_Fpet.RID.isin(df_Fpet_DX_62days_first_Fpet.RID.value_counts().head(9).index.values)][['RID','DX','RIGHT_AMYGDALA_SUVR']].sort_values(by='RID')

In [104]:
# df_Fpet_DX_62days_first_Fpet[df_Fpet_DX_62days_first_Fpet.RID.isin(df_Fpet_DX_62days_first_Fpet.RID.value_counts().head(9).index.values)][['RID','DX','RIGHT_AMYGDALA_SUVR']].groupby('DX').RIGHT_AMYGDALA_SUVR.mean()

In [105]:
df_Fpet_DX_62days_first_Fpet_NPID=df_Fpet_DX_62days_first_Fpet.merge(df_NPIQ[df_NPIQ_selcol_lt].rename(columns={'VISCODE2':'VISCODE2_dx'}),on=['RID','VISCODE2_dx'])
df_Fpet_DX_62days_first_Fpet_NPID.shape,df_Fpet_DX_62days_first_Fpet.shape

((6, 347), (1566, 341))

In [106]:
df_Fpet_DX_62days_first_Fpet_NPID=df_Fpet_DX_62days_first_Fpet.merge(df_NPIQ[df_NPIQ_selcol_lt].rename(columns={'VISCODE2':'VISCODE2_dx'}),on=['RID'])
df_Fpet_DX_62days_first_Fpet_NPID.shape,df_Fpet_DX_62days_first_Fpet.shape

((2877, 348), (1566, 341))

In [107]:
df_Fpet_DX_62days_first_Fpet_NPID[['RID','SCANDATE_dt']].drop_duplicates().shape

(981, 2)

In [108]:
df_Fpet_DX_62days_first_Fpet_NPID.groupby('NPID')['RIGHT_AMYGDALA_SUVR'].count()

NPID
-1.0       3
 0.0    2213
 1.0     373
 2.0       5
Name: RIGHT_AMYGDALA_SUVR, dtype: int64

In [109]:
df_Fpet_DX_62days_first_Fpet_NPID[df_Fpet_DX_62days_first_Fpet_NPID.NPID.isin([0,1])].groupby('NPID')['RIGHT_AMYGDALA_SUVR'].mean()

NPID
0.0    1.293255
1.0    1.300507
Name: RIGHT_AMYGDALA_SUVR, dtype: float64

In [110]:
df_Fpet_DX_62days_first_Fpet_NPID[df_Fpet_DX_62days_first_Fpet_NPID.NPID.isin([0,1])].groupby('NPID')['CTX_ENTORHINAL_SUVR'].mean()

NPID
0.0    1.228127
1.0    1.243933
Name: CTX_ENTORHINAL_SUVR, dtype: float64

In [111]:
df_Fpet_npiq_DX_62days_final3_Fpet.groupby('NPID')['CTX_ENTORHINAL_SUVR'].count()

NPID
0.0    169
1.0     12
Name: CTX_ENTORHINAL_SUVR, dtype: int64

In [112]:
df_Fpet_npiq_DX_62days_final3_Fpet.groupby('NPID')['CTX_ENTORHINAL_SUVR'].mean()

NPID
0.0    1.179609
1.0    1.189917
Name: CTX_ENTORHINAL_SUVR, dtype: float64

In [113]:
df_Fpet_npiq_DX_62days_final3_Fpet.groupby('NPID')['RIGHT_AMYGDALA_SUVR'].count()

NPID
0.0    160
1.0     11
Name: RIGHT_AMYGDALA_SUVR, dtype: int64

In [114]:
df_Fpet_npiq_DX_62days_final3_Fpet.groupby('NPID')['RIGHT_AMYGDALA_SUVR'].mean()

NPID
0.0    1.234587
1.0    1.236727
Name: RIGHT_AMYGDALA_SUVR, dtype: float64

In [153]:
df_Fpet_DX_62days_first_Fpet[(df_Fpet_DX_62days_first_Fpet['DX']=='MCI')& (df_Fpet_DX_62days_first_Fpet.RID.isin(df_NPIQ[df_NPIQ['NPID']==0].RID))]['CTX_ENTORHINAL_SUVR'].mean()

1.2780052631578946

In [151]:
df_Fpet_DX_62days_first_Fpet[(df_Fpet_DX_62days_first_Fpet['DX']=='MCI')& (df_Fpet_DX_62days_first_Fpet.RID.isin(df_NPIQ[df_NPIQ['NPID']==1].RID))]['CTX_ENTORHINAL_SUVR'].mean()

1.2533692307692308